## NPT-SALIVA TESTING, COVID 19 
### Affan Shoukat 
#### Center for Disease Modelling and Analysis

In [1]:
## npt-saliva-testing
using JuMP, Ipopt
using GLPK
using DataFrames 
using CSV 
using LsqFit
using Distributions
using OffsetArrays
using Optim
using Gnuplot
Gnuplot.options.verbose=false
pwd()
println(Gnuplot.gpversion())

const ll = CSV.File("data/likelihoods.csv") |> DataFrame! ; 
const sens_csv = CSV.File("data/sensitivity.csv", header=true) |> DataFrame! ;

5.2.0


In [2]:
TVAL_OFFSET = -15:40 # run time for the infection.

## read the likelihood file from the Ashcroft paper. 
function get_infectivity_curve(t, shape, rate, shift)
    dist = Gamma(shape, 1/rate)    
    pdfvals = pdf.(dist, t .+ shift) 
end

# we don't really use ashcroft 
function sample_ashcroft()
    # return shape, rate, shift from ashcroft's likelihood
    N = nrow(ll)
    rr = rand(1:N)  
    shape, rate, shift = ll[rr, :].shape, ll[rr, :].rate, ll[rr, :].shift 
end

function plot_ashcroft_curve(tvals)    
    #tvals = -15:25 ## 25 since the harvard sensitivity data only has 26 points
    randcurves = hcat([get_infectivity_curve(tvals, sample_ashcroft()...) for _=1:1000]...)
    ymeans_ashcroft = dropdims(mean(randcurves, dims=2), dims=2) 
    ylows_ashcroft = quantile.(eachrow(randcurves), [0.025])
    yhighs_ashcroft = quantile.(eachrow(randcurves), [0.975])
    return ymeans_ashcroft, ylows_ashcroft, yhighs_ashcroft
end


function get_nature_curve(tvals)
    #tvals = -15:25 ## 25 since the harvard sensitivity data only has 26 points
    shrash =  20.516508, 1.592124, 12.272481    # nature.
    ymeans_nature = get_infectivity_curve(tvals, shrash...) 
    ymeans_offset = OffsetArray(ymeans_nature, tvals)
end

ymeans = get_nature_curve(TVAL_OFFSET)
@gp "set term svg enhanced standalone mouse size 800,300" ## use jupyter
#@gp "set term png enhanced size 600,400" ## use jupyter
#@gp "set size 0.5,0.5" ## sets the size wrt to the canvas
@gp :- tit="Infectivity Curve" key="opaque" 
@gp :- TVAL_OFFSET ymeans[TVAL_OFFSET] "w l title 'nature' lw 3 lc rgb 'blue'"
#@gp :- tvals ylows_ashcroft yhighs_ashcroft "with filledcu title 'quantile' lw 2 lc rgb '#a1d99b' "
#@gp :- tvals ymeans_ashcroft "w l title 'ashcroft' lw 3 lc rgb 'black'"


In [13]:
# read sensitivity data 
# first column is digitized from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7151472/
# second column is digitized from https://faseb.onlinelibrary.wiley.com/doi/10.1096/fj.202001700RR
function get_sensitivity_data()   
    sens = Int64.(sens_csv.harvard[1:26] )./100  # harvard data includes day 0 as symptom onset + 25 days of post symptoms
    sens = OffsetArray(sens, 0:25) # offset the array for easier indexing in fitting process. has nothing to do with TVAL_OFFSET
    return sens
end
@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
@gp :- "set xlabel 'days post symptom onset'"
@gp :- "set ylabel 'positivity'"
@gp :- tit="Sensitivity" key="opaque" 
@gp :- 0:25 get_sensitivity_data()[0:25] "with points notitle pt 7 pointsize 0.65 lc rgb 'black'"
display(@gp)

In [4]:
# gompertz function
function gompertz(shift_value, tvals)
    xvals = tvals[1]:(tvals[end] + shift_value - 1)
     _gmp = @. exp(-1*exp((-xvals)))
    gmp = [_gmp[i+shift_value - 1] for i = 1:length(tvals)]
    shift_gmp = OffsetArray(gmp, tvals)
end

@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
@gp :- "set xlabel 'symptom onset'" 
@gp :- TVAL_OFFSET gompertz(2, TVAL_OFFSET)[TVAL_OFFSET] "with lines notitle"

In [5]:
gmp_array = [gompertz(i, TVAL_OFFSET) for i = 1:10]
@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
@gp :- "set xlabel 'symptom onset'"
for i in 1:length(gmp_array)
    @gp :- TVAL_OFFSET gmp_array[i][TVAL_OFFSET] "with lines title 'delay $i'"
end
@gp

In [6]:
# samples the start of infectious period
# returns a positive number, but denotes the number of days before symptom onset
function infect_start()
    lat_dist = truncated(LogNormal(1.434065, 0.6612), 3, 15)
    inc_period = Int.(round.(rand(lat_dist)))
    return inc_period .- 1 #infectiousness starts one day after incubation
end

histdata = hist([infect_start() for _ = 1:1000])
@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
@gp :- "set style histogram cluster gap 2"
@gp :- "set style fill solid border -1"
@gp :- "set boxwidth 0.9"
@gp :- histdata.bins histdata.counts "w histeps lc rgb 'red'"

We want to optimize the parameters of a modified hill function that maps the infectivity curve to the sensitivity of the test. In particular, 
$$
\min_{n, C} \sum_{t = 1}^{30} \left( s(t) - \left[ \frac{r(t)^n}{r(t)^n + C} \right] g_p(t) \right)^2
$$
where $s(t)$ is the sensitivity (digitized from [source](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7151472/)) and $g_p(t)$ is a shifted Gompertz function by $p$ days.

In [63]:
function get_sensitivity(inf_start)    
    sens = get_sensitivity_data()
    ngmp = gompertz(inf_start, TVAL_OFFSET)
    ymeans_offset = get_nature_curve(TVAL_OFFSET)   
   
    # define function to the minimized
    f_min(p) = sum((sens[0:25] .- p[3].*ymeans_offset[0:25].^p[1] ./ (ymeans_offset[0:25].^p[1] .+ p[2]) .* ngmp[0:25]).^2)
    # define the modified hill function
    @. f(x, p) = (p[3]*x^p[1]  / (x^p[1] + p[2])) *  ngmp
    optz =optimize(f_min, [1, 0.5, 0.5]; autodiff = :forward)
    p0 = Optim.minimizer(optz)
    #println(p0)
    myhill = f(ymeans_offset, p0)
    return myhill
end
get_sensitivity(4)

56-element OffsetArray(::Array{Float64,1}, -15:40) with eltype Float64 with indices -15:40:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 2.5761513102022777e-176
 2.0130697842831555e-65
 1.3520222327284072e-24
 1.4857134266316517e-9
 0.0005244232941119029
 0.05891408185803413
 0.3393908262469984
 ⋮
 0.11947288339325374
 0.10380674507043193
 0.08993293366292901
 0.07770383678338075
 0.0669706376875737
 0.05758724991444264
 0.04941327931687439
 0.042316121112099675
 0.03617232015891103
 0.03086832758971895
 0.026300781089380408
 0.022376423702700897

In [59]:
myhill = get_sensitivity(4) # plot sensitivity curve with infectiouness starting at i days
println("maximum: $(maximum(myhill))")

get_saliva_curve(myhill) = rand(Uniform(0.6, 0.9)) * myhill
    
saliva_curves = hcat([get_saliva_curve(myhill) for _ = 1:1000]...)
saliva_mean = dropdims(mean(saliva_curves, dims=2), dims=2)
slows = quantile.(eachrow(saliva_curves), [0.025])
shighs = quantile.(eachrow(saliva_curves), [0.975])

@gp "set term svg enhanced standalone mouse size 800,400" ## use jupyter
@gp :- tit="Fitted Hill Curve" key="opaque" 
#@gp :- TVAL_OFFSET ymeans_offset[TVAL_OFFSET] "with lines title 'rel infectivity'"
@gp :- TVAL_OFFSET slows shighs "with filledcu title 'quantile' lw 2 lc rgb '#a1d99b' "
@gp :- TVAL_OFFSET saliva_mean "with lines title 'saliva mean' lc rgb 'black' lw 3"
@gp :- TVAL_OFFSET myhill[TVAL_OFFSET] "with boxes title 'npt' lc rgb 'blue'"
@gp :- 0:25 get_sensitivity_data()[0:25] "with points title 'sensitivity' pt 7 pointsize 0.65 lc rgb 'orange'" 


[0.1352168239344195, 0.4142366056451548, 1.4685187133434492]
maximum: 0.9372222219594142


In [852]:
## testing algorithm
    # 30 day experiment_day (person becomes sick on each one of these days)
    # apply the test freq: (example: 5 -> (5, 10, 15, 20, 25, 30))
    # get the individiuals start of infectiousness period (truncate between -15 to 0) 
    #  for each day of the infectiousness period, individual has a sensitivity curve. 
    #  see what tests they are eligble for based on test frequency and end of infectiousness period
    # sample against the sensitivity curve.

# problem: total_infect_days is really long

In [64]:
function next_tests_probs(test_type=:np, inf_type=:all, testfreq = 7, exp_days = 30)
    #test_type=:np; inf_type=:all; testfreq = 5; exp_days = 20
    dayoftest = Tuple([collect(testfreq:testfreq:exp_days)...])

    days_positive = zeros(Float64, exp_days)   

    for t = 1:exp_days
        inf_start = infect_start()
        sensitivity = get_sensitivity(inf_start)
        
        total_infect_days = length(sensitivity) ## get the total infectiousness period 
        possible_tests = filter(x -> x > t && x < total_infect_days + t , dayoftest)

        # get the relative_infectivity index for the offset array
        rel_idx = -15 .+ (possible_tests .- t)
        if inf_type == :pre
            rel_idx = filter(x -> x < 0, rel_idx)
        end

         # see what happens on the day of the test and record that for t. 
        tot_fails = map(rel_idx) do x 
            infect = sensitivity[x]
            if test_type == :sal 
                infect = infect * rand(Uniform(0.604, 0.966))
            end
            failure = 1 - infect
        end
        any_l = length(tot_fails)
        prob_success = any_l > 0 ? 1 - reduce(*, tot_fails) : 0
        days_positive[t] = prob_success
    end
    return days_positive    
end
dp = next_tests_probs()
#@gp "set term svg enhanced standalone mouse size 600,400" ## use jupyter
#@gp :- tit="Test Results" key="opaque" 
#@gp :- 1:length(dp) dp "with boxes notitle"
#@gp :- [dayoftest[i] for i = 1:length(dayoftest)] [1,1,1,1] "with points title 'test day' pt 7 pointsize 0.65 lc rgb 'black'"


30-element Array{Float64,1}:
 0.9933604884607152
 0.9945221946757288
 0.9967920019912451
 0.9862933778598474
 0.9868715776977837
 0.9783927915795155
 0.9400975239883784
 0.963177638061199
 0.9899823358864392
 0.9482870789603101
 0.9880418470516077
 0.9275002718039037
 0.9080012836627647
 ⋮
 0.7697120032518538
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [68]:
N = 1000
exp_days = 60
testfreq = 14
dayoftest = Tuple([collect(testfreq:testfreq:exp_days)...])
results = zeros(Float64, exp_days, N)
for i = 1:N
    results[:, i] = next_tests_probs(:np, :pre, testfreq, exp_days)
end
totals = mean(results, dims=2);


In [70]:

# totals_filtered = totals[2*freq_of_test:(experiment_days - 2*freq_of_test)]#
#plot(totals[window], seriestype=:bar)
#mean(totals[meanwindow])
#quantile(totals[meanwindow], [0.025, 0.975])
#mean(totals_filtered)
#histogram(mean(results[:, 1:150], dims=1)')

@gp "set term svg enhanced standalone mouse size 800,600" ## use jupyter
@gp :- tit="Test Results" key="opaque" 
@gp :- "set xlabel 'day of infection'"
@gp :- 1:exp_days totals "with boxes notitle"
#@gp :- [dayoftest[i] for i = 1:length(dayoftest)] [1,1,1,1] "with boxes title 'test day' pt 7 lc rgb 'green'"

#mean(totals[14:42])